## PHYS 1600 HW1
Tim Zhao, B01307256, Feb 10, 2021

In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

In [ ]:
#General ODE solver with Runge-Kutta 4
def RK4(f,t0,t_f,step_size,f0):
    """
    f - RHS of ODE
    t0 - lower limit of independent variable
    t_f - upper limit of independent variable
    """
    
    # number of time steps, n must be an integer
    n  = int(np.ceil((t_f-t0)/dt))
    
    #initialize some empty lists to store our data and time steps
    Q_list = []
    t_list = []
    
    for ii in range(n):
        if ii == 0.0:
            # set initial conditions at time zero
            Q = Q0
            t = t0
        else :
            # !!!!!!!!!!!!!!!!!!!!!!   RK2 method here  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            
            k1 = dt*(-Q_list[ii-1]/R/C)
            k2 = dt*(-(Q_list[ii-1]+0.5*k1)/R/C)
            
            # update value of Q at each time step and step time by dt
            Q+=k2
            t+=dt
            
            
        # use append to add on to the end of a list
        Q_list.append(Q)
        t_list.append(t)
    
    # we turn the lists into numpy arrays before returning
    
    Q_list = np.array(Q_list)
    t_list = np.array(t_list)
    
    return t_list, Q_list
    
    
    
    
    
    
    